### This is a Multi-class classifier for different types of fresh and rotten fruits


1. Fresh Apple
2. Fresh Orange
3. Rotten Apple
4. Rotten Orange



### Downloading dataset from google drive

In [1]:
!gdown 1TXwVYM3K69ocfj1t8wCS31OjoA0ajlRl

Downloading...
From: https://drive.google.com/uc?id=1TXwVYM3K69ocfj1t8wCS31OjoA0ajlRl
To: /content/fruits.zip
100% 981M/981M [00:15<00:00, 64.9MB/s]


### Extracting zip file

In [2]:
import zipfile

local_zip = './fruits.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('.')
zip_ref.close()

### Setting directories

In [3]:
import os

train_dir = os.path.join('./fruits/train')
validation_dir = os.path.join('./fruits/validation')

### Initializing and compiling the model





In [4]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])


In [5]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Setting up the ImageDataGenerator

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalizing images
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
        train_dir,
        target_size=(300, 300),
        batch_size=126,
        class_mode='categorical')

validation_gen = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 300),
        batch_size=126,
        class_mode='categorical')



Found 5790 images belonging to 4 classes.
Found 1297 images belonging to 4 classes.


### Training the model

In [7]:
history = model.fit(
    train_gen,
    steps_per_epoch=20,
    epochs=25,
    verbose=1,
    validation_data=validation_gen,
    validation_steps=3)

Epoch 1/25
20/20 [==============================] - 48s 1s/step - loss: 1.7513 - accuracy: 0.2967 - val_loss: 1.2437 - val_accuracy: 0.4524
Epoch 2/25
20/20 [==============================] - 19s 918ms/step - loss: 1.3802 - accuracy: 0.4075 - val_loss: 1.1426 - val_accuracy: 0.5026
Epoch 3/25
20/20 [==============================] - 21s 1s/step - loss: 1.2224 - accuracy: 0.4599 - val_loss: 1.0930 - val_accuracy: 0.5556
Epoch 4/25
20/20 [==============================] - 18s 902ms/step - loss: 1.0841 - accuracy: 0.5742 - val_loss: 0.9335 - val_accuracy: 0.5979
Epoch 5/25
20/20 [==============================] - 18s 909ms/step - loss: 0.8890 - accuracy: 0.6786 - val_loss: 0.8271 - val_accuracy: 0.6296
Epoch 6/25
20/20 [==============================] - 21s 1s/step - loss: 0.7002 - accuracy: 0.7310 - val_loss: 0.8019 - val_accuracy: 0.7063
Epoch 7/25
20/20 [==============================] - 18s 910ms/step - loss: 0.7210 - accuracy: 0.7242 - val_loss: 0.7898 - val_accuracy: 0.6799
Epoch 8/

### You can upload any image here to test the model

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array


uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = load_img(path, target_size=(300, 300))
  y = img_to_array(img)
  y = np.expand_dims(y, axis=0)

  images = np.vstack([y])
  classes = model.predict(images, batch_size=1)
  print(fn)
  classes = classes.flatten()

  mx = 0.0
  mxi = classes.argmax()
  print(classes)
  if(mxi == 0):
    print("Fresh Apple")
  elif(mxi == 1):
    print("Fresh Orange")
  elif(mxi == 2):
    print("Rotten Apple")
  else:
    print("Rotten Orange")